In [1]:
from langchain.chat_models import init_chat_model
from langgraph.store.memory import InMemoryStore
from langchain_core.messages import SystemMessage

from dotenv import load_dotenv
from langgraph.graph import StateGraph, START, END, MessagesState
from langgraph.store.base import BaseStore 
from langchain_core.runnables import RunnableConfig

import uuid
from typing import List
from pydantic import BaseModel, Field

load_dotenv()

True

Chatbot Creating New Memories(without Duplication)

In [2]:
# 1) LTM store
store = InMemoryStore()

In [3]:
memory_llm = init_chat_model("gpt-4o")

In [4]:
class MemoryItem(BaseModel):
    text: str = Field(description="Atomic user memory as a short sentence")
    is_new: bool = Field(description="True if this memory is NEW ands should be stored. False if already know")

In [5]:
class MemoryDecision(BaseModel):
    should_write: bool = Field(description="Whether to store any memories")
    memories: List[MemoryItem] = Field(default_factory=list, description="Atomic user memories to store")

In [6]:
memory_extractor = memory_llm.with_structured_output(MemoryDecision)

In [13]:
MEMORY_PROMPT = """You are responsible for updating and maintaining accurate user memory.

CURRENT USER DETAILS (existing memories):
{user_details_content}


TASK:
- Review the user's latest message.
- Extract user-specific info worth storing long-term (identity, stable preferences, ongoing projects/goals).
- For each extracted item, set is_new=true ONLY if it adds NEW information compared to CURRENT USER DETAILS.
- If it is basically the same meaning as something already present, set is_new=false.
- Keep each memory as a short atomic sentence.
- No speculation; only facts stated by the user.
- If there is nothing memory-worthy, return an empty list.
"""

In [14]:
def chat_creates_memory_node(state: MessagesState, config: RunnableConfig, store: BaseStore):

    user_id = config["configurable"]["user_id"]

    namespace = ("user", user_id, "details")

    # A) Load existing memories
    existing_items = store.search(namespace)
    existing_texts = [it.value.get("data", "") for it in existing_items if it.value.get("data")]
    user_details_content = "\n".join(f"- {t}" for t in existing_texts) if existing_texts else "(empty)"

    # B) Latest user message
    last_text = state["messages"][-1]

    # C) LLM extracts memories + marks new vs duplicate
    decision: MemoryDecision = memory_extractor.invoke(
        [
            SystemMessage(content=MEMORY_PROMPT.format(user_details_content=user_details_content)),
            {"role": "user", "content": f"USER MESSAGE:\n{last_text}"},
        ]
    )

    # D) Store ONLY new memories
    if decision.should_write:
        for mem in decision.memories:
            if mem.is_new:
                store.put(namespace, str(uuid.uuid4()), {"data": mem.text})
                return {"messages": [{"role": "assistant", "content": "Noted."}]}
            else:
                return {"messages": [{"role": "assistant", "content": "Memory already exist, chill"}]}
    else:
        return {"messages": [{"role": "assistant", "content": "Memory not IMP"}]}


In [15]:
# 4) Build graph: START -> chat -> END

builder = StateGraph(MessagesState)
builder.add_node("chat", chat_creates_memory_node)
builder.add_edge(START, "chat")
builder.add_edge("chat", END)

graph = builder.compile(store=store)

In [16]:
config = {"configurable": {"user_id": "u1"}}

In [11]:
r1 = graph.invoke({"messages": [{"role": "user", "content": "My name is Aayushmaan"}]}, config)
print("Assistant:", r1["messages"][-1].content)

Assistant: Noted.


In [12]:
r1 = graph.invoke({"messages": [{"role": "user", "content": "My name is Aayushmaan"}]}, config)
print("Assistant:", r1["messages"][-1].content)

Assistant: Memory not IMP


In [17]:
items = store.search(("user", "u1", "details"))

for item in items:
    print(item.value['data'])

The user's name is Aayushmaan.
